<a href="https://colab.research.google.com/github/andrepmau/primeiros_passos/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
%pip -q install google-genai

In [38]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [39]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [40]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################

resposta = client.models.generate_content(
    model=MODEL_ID,
    contents = "Quando será a próxima imersão de IA com google gemini da Alura?",
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 Para saber quando será a próxima Imersão IA com Google Gemini da Alura, a melhor forma é verificar diretamente nos canais oficiais da Alura:

*   **Site da Alura:** Procure na seção de cursos ou na página de eventos e imersões.
*   **Redes Sociais da Alura:** Siga a Alura no LinkedIn, Instagram, Twitter (X) e Facebook, pois eles costumam anunciar novidades por lá.
*   **Comunidade da Alura:** Participe do Discord ou fórum da Alura para ficar por dentro dos anúncios e discussões sobre os cursos.

Assim, você terá a informação mais precisa e atualizada sobre a próxima Imersão IA com Google Gemini.

esta parte manda o programa procurar na base de dados existente quando será a próxima imersão

In [41]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools":[{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura já aconteceu. A mais recente edição teve sua live de abertura em 9 de maio de 2025, e as inscrições ficaram abertas até o dia 11 de maio.

A Imersão IA da Alura em parceria com o Google Gemini é um programa online e gratuito, onde os participantes aprendem a criar prompts, automatizar tarefas e desenvolver sistemas de agentes autônomos usando inteligência artificial. Ao final do curso, os participantes recebem um certificado Alura + Google.


como a próxima imersão não está na base de dados atual da ferramenta, é o incluir uma pesquisa ao próprio google para obter um resultado mais correto e é isso que esta parte do código fez.

In [42]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura', 'Alura Imersão IA Google Gemini']
Páginas utilizadas na resposta: youtube.com, alura.com.br



esta parte apenas cria esse botão abaixo para "confirmar" a pesquisa diretamente no google

In [43]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install google-adk

instalou o adk do google
se utilizasse a linha
%pip install -q google-adk
não apareceriam tantas linhas de execução, ficando apenas o final com a lista dos ip


In [44]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

código para declarar variáveis, chamar comandos e organizar textos.

In [45]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

código que não mostraram para que serve... executar e pronto

In [46]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

outra parte de código apenas para exibição

In [47]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model= "gemini-2.0-flash",
        description="Agente que busca informações no Google",
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisa. Sua tarefa á usar a ferramenta de busca do goole (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Fique em no máximo 5 lançamentos relevantes, com base na quantidade e no entusiamos das notícias sobre ele.
        Se o tema tiver poucas notíticas ou reações entusiasmadas, é possível que ele não seja tão relevante assim e pode
        ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos


    # Executa o agente

In [48]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramentda de busca do google (googe_search) para criar um plano de sobre quais são os pontos
        mais relevantes que poderíamos abordar em um post sobre cada um deles. Você também por usar o google (google_search) para encontrar mais informações
        sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e
        retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [49]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [50]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [51]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################

if not topico:
  print ("Você não digitou um tópico")
else:
  print("Maravilha! Então vamos criar o post sobre em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("--------------------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("--------------------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("--------------------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: inss
Maravilha! Então vamos criar o post sobre em {topico}

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais recentes e relevantes sobre o INSS, aqui estão algumas perguntas que podem ajudar na busca:
> 
> 
> Com base nas notícias mais recentes sobre o INSS, aqui estão os 5 lançamentos mais relevantes para maio de 2025:
> 
> 1.  **Reajuste dos Benefícios do INSS em 2025:** Os segurados do INSS que recebem acima de um salário mínimo tiveram um reajuste de 4,77% a partir de janeiro de 2025. Esse reajuste corresponde ao Índice Nacional de Preços ao Consumidor (INPC) de 2024. O teto dos benefícios previdenciários aumentou para R$ 8.157,41. Para quem recebe salário mínimo, o reajuste foi de 7,5%, elevando o valor para R$ 1.518,00.
> 2.  **Calendário de Pagamentos do INSS:** O INSS divulgou o calendário de pagamentos para 2025. Em maio de 2025, os pagamentos serão realizados de 26 de maio a 6 de junho. Os pagamentos são feitos de forma escalonada, com prioridade para quem recebe até um salário mínimo.
> 3.  **Antecipação do 13º Salário:** Aposentados e pensionistas do INSS receberão a segunda parcela do 13º salário a partir de 26 de maio, juntamente com o benefício regular. O pagamento da segunda parcela do 13º salário será efetuado entre 26 de maio e 6 de junho, conforme o número final do cartão de benefício.
> 4.  **Novas Regras de Aposentadoria:** Em 2025, haverá um ajuste na idade para pedir aposentadoria devido às regras de transição da Reforma da Previdência. Para mulheres, a idade mínima será de 59 anos, com 30 anos de contribuição, e para homens, 64 anos, com 35 anos de contribuição.
> 5.  **Ressarcimento de Descontos Indevidos:** O INSS começou a ressarcir aposentados e pensionistas afetados por descontos associativos indevidos a partir de 26 de maio. O valor total a ser devolvido é de R$ 292.699.250,33, referente às mensalidades descontadas em abril. Os segurados podem consultar informações sobre os descontos e valores no aplicativo Meu INSS.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos mais recentes sobre o INSS para maio de 2025, vou criar um plano de conteúdo para cada um deles, identificando os pontos mais relevantes e buscando informações adicionais para aprofundar os temas. Em seguida, escolherei o tema mais relevante e detalharei um plano para um post.
> 
> **1. Reajuste dos Benefícios do INSS em 2025:**
> 
> *   **Pontos Relevantes:**
>     *   Percentual de reajuste para quem recebe acima do salário mínimo (4,77%).
>     *   Percentual de reajuste para quem recebe o salário mínimo (7,5%).
>     *   Novo valor do salário mínimo (R$ 1.518,00).
>     *   Novo teto dos benefícios previdenciários (R$ 8.157,41).
>     *   Correspondência do reajuste com o INPC de 2024.
> *   **Informações Adicionais a Buscar:**
>     *   Impacto do reajuste na vida dos beneficiários.
>     *   Comparativo com a inflação e o custo de vida.
>     *   Como o reajuste é calculado.
>     *   Tabela detalhada com os novos valores dos benefícios.
> 
> **2. Calendário de Pagamentos do INSS:**
> 
> *   **Pontos Relevantes:**
>     *   Datas de pagamento para maio de 2025 (26 de maio a 6 de junho).
>     *   Escalonamento dos pagamentos conforme o valor do benefício.
>     *   Prioridade para quem recebe até um salário mínimo.
> *   **Informações Adicionais a Buscar:**
>     *   Calendário completo de pagamentos para todo o ano de 2025.
>     *   Como consultar a data de pagamento específica no site ou app Meu INSS.
>     *   Informações sobre canais de recebimento do benefício.
> 
> **3. Antecipação do 13º Salário:**
> 
> *   **Pontos Relevantes:**
>     *   Pagamento da segunda parcela do 13º salário a partir de 26 de maio.
>     *   Quem tem direito ao 13º salário (aposentados e pensionistas).
>     *   Datas de pagamento da segunda parcela (26 de maio a 6 de junho).
> *   **Informações Adicionais a Buscar:**
>     *   Como o valor do 13º salário é calculado.
>     *   Imposto de Renda sobre o 13º salário.
>     *   Impacto econômico da antecipação do 13º salário.
>     *   Como o pagamento é realizado, incluindo detalhes sobre o número final do cartão de benefício e o escalonamento.
> 
> **4. Novas Regras de Aposentadoria:**
> 
> *   **Pontos Relevantes:**
>     *   Ajuste na idade mínima para aposentadoria em 2025.
>     *   Idade mínima para mulheres (59 anos) e tempo de contribuição (30 anos).
>     *   Idade mínima para homens (64 anos) e tempo de contribuição (35 anos).
>     *   Regras de transição da Reforma da Previdência.
> *   **Informações Adicionais a Buscar:**
>     *   Detalhes sobre as diferentes regras de transição.
>     *   Simulador de aposentadoria para calcular o tempo restante.
>     *   Impacto das novas regras na vida dos trabalhadores.
>     *   Como planejar a aposentadoria com as novas regras.
> 
> **5. Ressarcimento de Descontos Indevidos:**
> 
> *   **Pontos Relevantes:**
>     *   INSS começou a ressarcir descontos indevidos a partir de 26 de maio.
>     *   Valor total a ser devolvido (R$ 292.699.250,33).
>     *   Referência aos descontos de abril.
>     *   Como consultar informações sobre os descontos no app Meu INSS.
> *   **Informações Adicionais a Buscar:**
>     *   Quem tem direito ao ressarcimento.
>     *   Como verificar se você tem direito ao ressarcimento.
>     *   Prazos para o ressarcimento.
>     *   O que fazer se você não receber o ressarcimento.
>     *   Canais de atendimento para dúvidas e reclamações.
>     *   Quais foram as associações responsáveis pelos descontos indevidos.
>     *   Como evitar cair em golpes relacionados a descontos indevidos.
> 
> Agora, vamos buscar mais informações sobre esses temas usando a ferramenta de busca do Google:
> 
> Após analisar os lançamentos e as informações adicionais buscadas, o tema mais relevante para um post no momento é o **Ressarcimento de Descontos Indevidos**. A grande procura por informações sobre o tema e o alto número de queixas demonstram a urgência e o impacto dessa questão na vida dos segurados.
> 
> **Tema:** Ressarcimento de Descontos Indevidos do INSS em 2025
> 
> **Pontos Relevantes:**
> 
> *   **O que são os descontos indevidos:** Cobranças realizadas por associações e entidades sem a autorização do beneficiário.
> *   **Quem tem direito ao ressarcimento:** Aposentados e pensionistas do INSS que identificarem descontos não autorizados em seus benefícios.
> *   **Valor total a ser devolvido:** R$ 292.699.250,33.
> *   **Como verificar se você tem direito:** Consultar o extrato no aplicativo Meu INSS ou ligar para a Central 135.
> *   **Prazos para o ressarcimento:** Os ressarcimentos iniciais serão efetuados entre 26 de maio e 6 de junho, referentes aos descontos de abril.
> *   **O que fazer se você não receber o ressarcimento:** Registrar reclamação na ouvidoria do INSS e no Procon.
> *   **Golpes:** Alerta sobre golpes relacionados ao ressarcimento e a importância de não fornecer dados bancários por telefone ou WhatsApp.
> *   **O que acontece se a associação não pagar:** O caso será encaminhado à Advocacia Geral da União (AGU).
> 
> **Plano de Conteúdo para o Post:**
> 
> 1.  **Título:** INSS Devolve Dinheiro: Descubra se Você Tem Direito ao Ressarcimento de Descontos Indevidos!
> 
> 2.  **Introdução:**
>     *   Contextualizar a situação dos descontos indevidos e a Operação Sem Desconto.
>     *   Informar sobre o início do ressarcimento pelo INSS.
>     *   Destacar a importância de verificar o extrato e reivindicar o ressarcimento.
> 
> 3.  **O Que São Descontos Indevidos?**
>     *   Explicar detalhadamente o que são os descontos indevidos e como eles ocorrem.
>     *   Mencionar as associações e entidades envolvidas.
> 
> 4.  **Quem Tem Direito ao Ressarcimento?**
>     *   Especificar quem tem direito ao ressarcimento e como identificar se você é elegível.
>     *   Informar sobre a devolução para herdeiros de segurados falecidos.
> 
> 5.  **Como Verificar Se Você Tem Direito ao Ressarcimento?**
>     *   Passo a passo detalhado de como consultar o extrato no aplicativo Meu INSS.
>     *   Instruções sobre como entrar em contato com a Central 135.
> 
> 6.  **Prazos e Como Receber o Ressarcimento:**
>     *   Informar sobre o calendário de pagamentos e os prazos para o ressarcimento.
>     *   Esclarecer como o valor será devolvido (depósito na conta bancária).
>     *   O que fazer caso o valor não seja ressarcido no prazo.
> 
> 7.  **Alerta de Golpes:**
>     *   Reforçar os cuidados para evitar golpes relacionados ao ressarcimento.
>     *   Orientar a nunca fornecer dados bancários por telefone ou WhatsApp.
> 
> 8.  **O Que Fazer Se Você Não Receber o Ressarcimento?**
>     *   Instruções sobre como registrar reclamação na ouvidoria do INSS e no Procon.
>     *   Informar sobre a possibilidade de buscar auxílio jurídico.
> 
> 9.  **Conclusão:**
>     *   Reafirmar a importância de verificar o extrato e reivindicar seus direitos.
>     *   Incentivar o compartilhamento da informação para que mais pessoas tenham acesso.
>     *   Ressaltar que o INSS está tomando medidas para combater os descontos indevidos.
> 
> Este plano de conteúdo visa fornecer informações claras e acessíveis para que os segurados do INSS possam verificar se têm direito ao ressarcimento de descontos indevidos e como proceder para garantir seus direitos.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🚨 ALERTA INSS: Dinheiro EXTRA na sua conta? 🚨
> 
> Sabia que o INSS está devolvendo grana de descontos indevidos? 💰 Se você é aposentado ou pensionista, pode ter direito a essa grana!
> 
> 🔍 **O que rolou?** Algumas associações fizeram cobranças sem autorização e, agora, a grana tá voltando pra você!
> 
> 🤔 **Como saber se tenho direito?** É fácil!
> 1️⃣ Acesse o app Meu INSS
> 2️⃣ Consulte seu extrato e veja se tem algum desconto estranho
> 3️⃣ Se tiver, corra atrás do seu ressarcimento!
> 
> 🗓️ **Quando recebo?** Fique de olho! Os pagamentos já começaram, mas se não receber, não se desespere! Reclame na ouvidoria do INSS e no Procon.
> 
> ⚠️ **Cuidado com golpes!** ⚠️ NUNCA forneça seus dados bancários por telefone ou WhatsApp. O INSS não pede essas informações!
> 
> Marque um amigo aposentado pra ficar ligado nessa novidade! 😉
> 
> #INSS #Ressarcimento #SeusDireitos #Alura
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!
> 


--------------------------------------------------------------
